#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

In [1]:
#put here your ``numero di matricola''
numero_di_matricola = 1 # COMPLETE

## Insert your ID number ("numero di matricola") below

In [2]:
#put here your ``numero di matricola''
numero_di_matricola = 1 # COMPLETE

In [3]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [4]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [5]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $2/3$ of all samples, the one for choosing among different models will consist of $1/6$ of all samples, while the other part consists of the remaining $1/6$-th of all samples.

In [6]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [7]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [8]:
#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
mlp = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola )

#let's learn the model on training data
mlp.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp.intercepts_)

Training error:  0.2639482635219801
Validation error:  0.3040483726273231
[array([[-214.7718307 ],
       [ 269.23441249],
       [ 524.19253543],
       [ -60.69733143],
       [   3.83450459],
       [ 710.96734997],
       [ 294.66380839],
       [ 136.73475436],
       [ 816.14054714],
       [ 493.71841477],
       [ 163.47786773],
       [-582.05648627],
       [  38.05704654],
       [-203.48545654],
       [ 600.52470753],
       [-142.07163869],
       [ 147.10066717],
       [ -27.00076473]]), array([[141.06662337]])]
[array([3796.20300252]), array([-49.17158744])]


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [9]:
  from sklearn import linear_model
  LR = linear_model.LinearRegression()
  LR.fit(Xtrain_scaled, Ytrain)
  print("Training error: ", 1. - LR.score(Xtrain_scaled, Ytrain))
  print("Validation error: ", 1. - LR.score(Xval_scaled, Yval))
  print(LR.coef_)
  print(LR.intercept_)

Training error:  0.2653594216072852
Validation error:  0.3115400506517969
[-31303.71909156  35848.45081517  74506.78099995  -8012.41104949
    671.23713588 100205.53195594  41671.19028923  19507.84532115
 111331.50566184  69959.22677526  23468.73219785 -78236.93092911
   6535.34729956 -28197.21476235  83701.76486765 -21647.26671149
  22056.22833416  -2002.69401407]
536831.9203413766


Is there a way to make a NN network learn a linear model?

Let's first check what is the loss used by MLPRegressor...

In [10]:
#let's write the code to learn a linear model with NN: how? 

#let's define the model
mlp_lr = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola, activation='identity')
#let's learn the model on training data
mlp_lr.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_lr.score(Xtrain_scaled, Ytrain))
#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_lr.score(Xval_scaled, Yval))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr.intercepts_)

Training error:  0.26535942166590454
Validation error:  0.3115390658284922
[array([[  51.55070235],
       [ -59.02846704],
       [-122.96939596],
       [  13.19466306],
       [  -1.10694448],
       [-165.016906  ],
       [ -68.62491747],
       [ -32.12596414],
       [-183.34020234],
       [-114.96558392],
       [ -38.51551396],
       [ 128.83877577],
       [ -10.76237595],
       [  46.43749972],
       [-137.83874569],
       [  35.64844107],
       [ -36.32214078],
       [   3.29740834]]), array([[-607.24280969]])]
[array([-883.44724722]), array([365.29153421])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [11]:
#let's write the code to learn a linear model with NN: how? 
#let's define the model
mlp_lr_noreg = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola, activation='identity', alpha = 1e-20)
#let's learn the model on training data
mlp_lr_noreg.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_lr_noreg.score(Xtrain_scaled, Ytrain))
#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_lr_noreg.score(Xval_scaled, Yval))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr_noreg.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr_noreg.intercepts_)

Training error:  0.26535942166590454
Validation error:  0.3115390658282986
[array([[  51.55070235],
       [ -59.02846704],
       [-122.96939596],
       [  13.19466306],
       [  -1.10694448],
       [-165.016906  ],
       [ -68.62491747],
       [ -32.12596414],
       [-183.34020234],
       [-114.96558392],
       [ -38.51551396],
       [ 128.83877577],
       [ -10.76237595],
       [  46.43749972],
       [-137.83874569],
       [  35.64844107],
       [ -36.32214078],
       [   3.29740834]]), array([[-607.24280969]])]
[array([-883.44724722]), array([365.29153421])]


## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [12]:
#let's build a NN with 2 nodes in the only hidden layer

#let's define the model
mlp_1h2n = MLPRegressor(hidden_layer_sizes=(2, ), solver='lbfgs', random_state=numero_di_matricola )
#let's learn the model on training data
mlp_1h2n.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_1h2n.score(Xtrain_scaled, Ytrain))
#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_1h2n.score(Xval_scaled, Yval))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h2n.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h2n.intercepts_)

Training error:  0.18062155311270378
Validation error:  0.20722328931237033
[array([[  90.89730895,  -33.21920939],
       [ 120.36307534,   39.22262673],
       [  85.8874491 ,   73.0254267 ],
       [-265.45693741,   28.17956363],
       [ -30.96979713,   17.95968057],
       [ 197.45850144,   26.23067006],
       [  34.85107625,   37.58830115],
       [  96.88535711,   25.80784857],
       [ 312.45376344,  133.02403964],
       [  85.182243  ,   68.78365263],
       [  19.21442949,   23.42565799],
       [-217.79948328,  -81.00705803],
       [  -3.4218254 ,   20.18457634],
       [-300.95191675,  -26.38662247],
       [ 305.56099842,  144.57600922],
       [-463.55776786,  -16.39269194],
       [ 194.88158085,   52.59667455],
       [-246.78646482,  -11.06097008]]), array([[614.22767597],
       [548.87463254]])]
[array([-1049.39355061,   897.60825833]), array([725.96915573])]


Let's build a NN with 5 nodes in the only hidden layer

In [13]:
#let's build a NN with 5 nodes in the only hidden layer

#let's define the model
mlp_1h5n = MLPRegressor(hidden_layer_sizes=(5, ), solver='lbfgs', random_state=numero_di_matricola )
#mlp_1h5n = MLPRegressor(hidden_layer_sizes=(5, ), solver='lbfgs', random_state=numero_di_matricola,  max_iter=2000 )
#let's learn the model on training data
mlp_1h5n.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_1h5n.score(Xtrain_scaled, Ytrain))
#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_1h5n.score(Xval_scaled, Yval))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h5n.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h5n.intercepts_)

Training error:  0.16243223694737163
Validation error:  0.2131543382151624
[array([[ -129.9858601 ,   139.42036522,    98.35416973,   176.56500054,
          -60.16392052],
       [  277.00674902,   392.69059344,   192.61952077,   141.99964615,
           -2.47161108],
       [ -354.01045518,   318.31484048,   222.45296278,  -283.38605503,
          263.12459659],
       [ -127.68253326,   -20.58252199,  -689.03464463,    91.65258382,
           65.34464381],
       [  791.5859972 ,  -862.86856501,   176.36615457,   -36.48657049,
         -104.39386936],
       [ -361.19484633,   627.93076147,   490.65579641,  -479.16472011,
          -81.88251344],
       [ -562.88115896,   609.66825501,   -14.16595317,  -425.46311293,
          190.41234934],
       [   98.51541173,   718.59588791,   148.04906458,   -54.37854108,
           29.04992849],
       [ -359.56570823,   643.9300707 ,   539.2065387 ,    54.44387611,
          402.64849267],
       [ -439.92935551,   523.97678956,   258.72623

/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [14]:
#let's build a NN with 10 nodes in the only hidden layer

#let's define the model
mlp_1h10n = MLPRegressor(hidden_layer_sizes=(10, ), solver='lbfgs', random_state=numero_di_matricola )
#mlp_1h5n = MLPRegressor(hidden_layer_sizes=(5, ), solver='lbfgs', random_state=numero_di_matricola,  max_iter=2000 )
#let's learn the model on training data
mlp_1h10n.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_1h10n.score(Xtrain_scaled, Ytrain))
#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_1h10n.score(Xval_scaled, Yval))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h10n.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h10n.intercepts_)

Training error:  0.12262425170612223
Validation error:  0.2910429221424544
[array([[ 114.31587761,  -32.96282606,  -66.44887627,   63.50956523,
          51.21158679,  -29.10030873,  -10.11603783,    5.25035985,
         -47.31033131,  167.1851757 ],
       [  45.45160946,  -43.227364  ,   82.49313116,   82.22816724,
         -82.99676011,  402.85251214,  290.53083434,  -16.7131106 ,
         112.47564565, -146.53125771],
       [-128.98439775,   27.12290235,  242.66720383,   20.4205301 ,
        -182.29177789,  189.40480387, -180.04524706,   -1.90189224,
          91.79363639, -153.70219132],
       [ 186.31036201,   60.57463242,   61.55537554, -199.14285605,
         -83.43801207,  -19.54385485, -186.48552339, -135.32289153,
         -44.41015977, -489.96681947],
       [-517.60453465,  543.122291  , -185.35713446,   97.13578896,
         153.36154446,  -24.26065817, -158.83078416,  187.97680285,
         169.29208402, -543.94360008],
       [ 287.8750709 , -192.42447294,  -79.225886

/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [15]:
#let's build a NN with 100 nodes in the only hidden layer

#let's define the model
mlp_1h100n = MLPRegressor(hidden_layer_sizes=(100, ), solver='lbfgs', random_state=numero_di_matricola )
#let's learn the model on training data
mlp_1h100n.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_1h100n.score(Xtrain_scaled, Ytrain))
#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_1h100n.score(Xval_scaled, Yval))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h100n.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h100n.intercepts_)

Training error:  0.030652398971860007
Validation error:  0.5460391503923037
[array([[  96.52005711,  -24.82150669,   75.00925839, ..., -110.35839702,
         184.74830304,  -91.85519834],
       [  -0.60387449,   65.7633649 ,   90.53889604, ...,   38.32321663,
         -28.71587054,  -19.72895332],
       [ -80.20461506,  -77.69632347,  105.32762817, ...,   29.79151667,
         257.34110208,   47.50929344],
       ...,
       [  47.36253499,  -68.60472001,   68.572011  , ..., -379.12326438,
          62.74548799,  -45.96891651],
       [ -48.39542675,   79.18929692,  157.43803322, ..., -116.8667761 ,
         -81.18391147,  -51.99774453],
       [   9.86044893, -174.96269367,   42.96902508, ...,   64.81285698,
         -10.38845328,    4.28225502]]), array([[ 122.57559456],
       [  38.74216552],
       [ -31.29397729],
       [ -48.86074317],
       [ 214.32260913],
       [  68.41260526],
       [  58.56603595],
       [ 288.49282292],
       [-251.74979744],
       [-212.74343839

/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [16]:
#let's build a NN with 2 hidden layers, 1 node each
#let's define the model
mlp_2h1n1n = MLPRegressor(hidden_layer_sizes=(1,1 ), solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
#mlp_2h2n2n = MLPRegressor(hidden_layer_sizes=(1,1 ), solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)
#let's learn the model on training data
mlp_2h1n1n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h1n1n.score(Xtrain_scaled,Ytrain)))
#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h1n1n.score(Xval_scaled,Yval)))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h1n1n.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h1n1n.intercepts_)

1 - coefficient of determination on training data:0.23817894410095009
1 - coefficient of determination on validation data:0.271308623127011
[array([[ -36.83453526],
       [ 108.06974253],
       [ 187.18744894],
       [ -51.50055654],
       [ -10.18577287],
       [ 220.21692028],
       [  84.78177649],
       [  68.56810517],
       [ 336.0407839 ],
       [ 182.05200714],
       [  48.66855808],
       [-228.44799589],
       [  19.5779236 ],
       [-154.3285914 ],
       [ 318.75789447],
       [-215.36423594],
       [ 103.96531861],
       [ -36.25231323]]), array([[1.07452441]]), array([[426.16415211]])]
[array([218.55101166]), array([755.90527631]), array([486.76904942])]


/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 2 nodes each

In [17]:
#let's build a NN with 2 layers, 2 nodes each

#let's define the model
mlp_2h2n = MLPRegressor(hidden_layer_sizes=(2,2, ), solver='lbfgs', random_state=numero_di_matricola )
#let's learn the model on training data
mlp_2h2n.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_2h2n.score(Xtrain_scaled, Ytrain))
#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_2h2n.score(Xval_scaled, Yval))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h2n.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h2n.intercepts_)

Training error:  0.21257591478722637
Validation error:  0.2698618692593906
[array([[ -3.74716396,   1.64207598],
       [ 10.40429154,  -6.96593909],
       [ 20.90286001, -27.2156469 ],
       [ -8.26926967,  12.06986195],
       [ -4.18140748,  14.66619964],
       [ 21.44376613, -13.8362104 ],
       [  8.89850908,  -8.71615806],
       [  7.99615918,  -7.88367252],
       [ 37.78297323, -40.85134948],
       [ 20.32196503, -26.17577388],
       [  5.71862963,  -6.63056441],
       [-26.84817542,  34.09887519],
       [  0.97451326,   2.64322041],
       [-15.06222276,  20.03902168],
       [ 24.82710507,  -9.26489523],
       [-18.12301164,  24.87504215],
       [  5.24213365,   7.14013654],
       [ -3.09937223,   8.02804359]]), array([[ -2.35165244,  63.81081038],
       [-11.52752663,  25.78116573]]), array([[ 1.87289108],
       [75.57987877]])]
[array([81.76610784, 69.66510019]), array([ -10.98075336, -510.17173588]), array([145.53050918])]


/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 10 nodes each

In [18]:
#let's build a NN with 2 layers, 10 nodes each

#let's build a NN with 2 layers, 10 nodes each
#let's define the model
mlp_2h10n10n = MLPRegressor(hidden_layer_sizes=(10,10 ), solver='lbfgs', random_state=numero_di_matricola)
#let's learn the model on training data
mlp_2h10n10n.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h10n10n.score(Xtrain_scaled,Ytrain)))
#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h10n10n.score(Xval_scaled,Yval)))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h10n10n.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h10n10n.intercepts_)

1 - coefficient of determination on training data:0.07224101465295363
1 - coefficient of determination on validation data:0.28382419236682566
[array([[ 1.90000073e+00,  6.74612309e+00,  8.72469238e+00,
        -2.98794529e-01, -6.67200317e+00, -2.70416738e+01,
        -9.15761939e+00, -1.69076066e+01, -4.22345606e+00,
         1.59918350e+01],
       [ 3.73355079e+00,  3.07579299e+01,  2.66922788e+01,
        -1.24439961e+00, -6.98036637e+00,  4.75155681e+01,
        -2.13747798e+01,  8.54998125e+00, -6.05943702e+00,
        -3.02640243e+00],
       [-2.70532448e+01,  1.29394167e+01,  1.10042754e+01,
        -1.16887972e+01, -1.47391004e+01,  6.07380288e+00,
         2.03918771e-01,  3.57268474e+01, -4.11211550e+01,
        -3.83539312e+00],
       [-2.90036499e+00,  6.47216320e-01,  1.12451960e+01,
        -1.72459547e+01, -1.90694154e+01, -3.54493789e+00,
        -8.19023867e+00, -4.89369683e+00,  7.43076707e+00,
         8.89034567e-01],
       [-4.22260639e+01,  3.36157059e+01, -2.

/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 100 nodes each

In [19]:
#let's build a NN with 2 layers, 100 nodes each

#let's define the model
mlp_2h100n100n = MLPRegressor(hidden_layer_sizes=(100,100 ), solver='lbfgs', random_state=numero_di_matricola)
#let's learn the model on training data
mlp_2h100n100n.fit(Xtrain_scaled, Ytrain)
#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h100n100n.score(Xtrain_scaled,Ytrain)))
#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h100n100n.score(Xval_scaled,Yval)))
#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h100n100n.coefs_)
#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h100n100n.intercepts_)

1 - coefficient of determination on training data:0.022759034636666953
1 - coefficient of determination on validation data:0.32728890124079935
[array([[  7.9243215 ,   1.59402075,  -0.41935646, ...,  -2.88479191,
        -12.68433515,  -0.56791349],
       [ -5.14122677,   1.67120972,   2.34593065, ...,  -7.18489743,
          5.68689912,  -5.49484271],
       [ -0.95919661,  -0.22908913,   3.53009396, ...,   1.65053641,
         -7.00077137,  -5.01617861],
       ...,
       [  1.09846296,  -2.81892098,  -4.73774699, ...,  29.30226112,
        -18.6945582 ,   2.16607803],
       [ -1.84375164,  -1.6013884 ,  -8.04435393, ...,  -0.99008174,
         24.87864137,  -4.59980221],
       [-20.51304209,  -3.02467941,   4.17344858, ...,   3.53621329,
         -1.89547898,   2.66945091]]), array([[-2.43500782,  0.28863306, -1.04153725, ...,  0.17382546,
         0.19378034, -0.55900835],
       [-2.19002595,  0.18655725, -0.02481516, ...,  0.0567985 ,
         0.25483638,  0.53449524],
      

/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [20]:
from sklearn.model_selection import GridSearchCV
mlp_cv = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,21)],
                'activation': ['relu'],
                'solver': ['lbfgs'], 
                'random_state': [numero_di_matricola]
            }
mlp_GS = GridSearchCV(mlp_cv, param_grid=param_grid, cv=5, verbose=True)
mlp_GS.fit(Xtrain_and_val_scaled, Ytrain_and_val)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/daniel/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1)

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu'],
                         'hidden_layer_sizes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                11, 12, 13, 14, 15, 16, 17, 18,
                                                19, 20],
                         'random_state': [1], 'solver': ['lbfgs']},
             verbose=True)

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [21]:
#let's print the best model according to grid search
print("Best model: ",mlp_GS.best_estimator_)

#let's print the error 1-R^2 for the best model
print("Error (1-R^2) of best model: ",1. - mlp_GS.best_score_)

Best model:  MLPRegressor(hidden_layer_sizes=5, random_state=1, solver='lbfgs')
Error (1-R^2) of best model:  0.2172067169532207


Let compare the error of the best NN on train and validation and on test data.

In [22]:
print("Error best model on train and validation: ",1. - mlp_GS.best_estimator_.score(Xtrain_and_val_scaled,Ytrain_and_val))
print("Error best model on test data: ",1. - mlp_GS.best_estimator_.score(Xtest_scaled,Ytest))

Error best model on train and validation:  0.15045690876034645
Error best model on test data:  0.23033103141760192


Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [24]:
#LR the linear regression model
LR = linear_model.LinearRegression()
#fit the model on training data
LR.fit(Xtrain_and_val_scaled, Ytrain_and_val)
print("1 - coefficient of determination on training data:"+str(1 - LR.score(Xtrain_and_val_scaled,Ytrain_and_val)))
print("1 - coefficient of determination on test data:"+str(1 - LR.score(Xtest_scaled,Ytest)))

1 - coefficient of determination on training data:0.2715568577139226
1 - coefficient of determination on test data:0.33736448787674667


Note: MLPRegressor has several other parameters!